In [ ]:
# ! pip install trulens_eval
# #! pip install chromadb
# ! pip install openai
# ! pip install llama_index
! pip install trulens_eval==0.19.2 openai==1.3.7 llama_index

In [ ]:
import os
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
#!!! TODO: inserire i parametri di accesso e di configurazione del dataset di evaluation !!!
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

API_KEY = "51166339f2f3498eb48e07a1c57a24b9"

os.environ["OPENAI_API_VERSION"] = "2023-07-01-preview"
os.environ["AZURE_OPENAI_API_KEY"] = API_KEY
os.environ["AZURE_OPENAI_API_BASE"] = "https://oaimfuccilo.openai.azure.com/"
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://oaimfuccilo.openai.azure.com/openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-07-01-preview"

deployment_name = "gpt-35-turbo-16k"
TestName = "MioTest"
cleanDB = True

#il dataset delle registrazioni dei test, in formato csv, che contenga le colonne: domanda, risposta, contesto
dataset_url = "test_dataset.csv"
dataset_delim = ";"
#se il dataset contiene l'header
dataset_header = True
#la posizione delle colonne nel dataset
dataset_column_question = 1
dataset_column_context = 3
dataset_column_response = 2


In [ ]:
from trulens_eval import Tru
from trulens_eval.tru_custom_app import instrument
tru = Tru()
#tru = Tru(database_url="sqlite:///mydb.sqlite")

In [ ]:
if cleanDB:
    tru.reset_database()

In [ ]:
import csv

#il dataset delle registrazioni dei test
DBEval = list()

with open(dataset_url) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=dataset_delim)
    DBEval = list(csv_reader)

In [ ]:
#il wrapper all'applicazione RAG
class RAG_app_facade:
    @instrument
    def retrieve(self, query: str, **kwargs) -> list:
        """
        Retrieve relevant text from index store.
        """
        row = kwargs.get('eval_row', 1)
        result = DBEval[row][dataset_column_context]
        
        return result

    @instrument
    def generate_completion(self, query: str, context_str: list, **kwargs) -> str:
        """
        Generate answer from context.
        """
        row = kwargs.get('eval_row', 1)
        completion = DBEval[row][dataset_column_response]
        return completion

    @instrument
    def query(self, query: str, **kwargs) -> str:
        row = kwargs.get('eval_row', 1)
        context_str = self.retrieve(query, eval_row=row)
        completion = self.generate_completion(query, context_str, eval_row=row)
        return completion

rag = RAG_app_facade()

In [ ]:
#Le funzioni di feedback da usare per valutare le risposte: https://www.trulens.org/trulens_eval/function_definitions/
from trulens_eval import Feedback, Select
from trulens_eval.feedback import Groundedness
from trulens_eval.feedback.provider.openai import AzureOpenAI as fOpenAI


import numpy as np

# Initialize provider class
fopenai = fOpenAI(deployment_name=deployment_name, api_key=os.getenv("AZURE_OPENAI_API_KEY"))

grounded = Groundedness(groundedness_provider=fopenai)

# Define a groundedness feedback function
f_groundedness = (
    Feedback(grounded.groundedness_measure_with_cot_reasons, name = "Groundedness")
    .on(Select.RecordCalls.retrieve.rets.collect())
    .on_output()
    .aggregate(grounded.grounded_statements_aggregator)
)

# Question/answer relevance between overall question and answer.
f_qa_relevance = (
    Feedback(fopenai.relevance_with_cot_reasons, name = "Answer Relevance")
    .on(Select.RecordCalls.retrieve.args.query)
    .on_output()
)

# Question/statement relevance between question and each context chunk.
f_context_relevance = (
    Feedback(fopenai.qs_relevance_with_cot_reasons, name = "Context Relevance")
    .on(Select.RecordCalls.retrieve.args.query)
    .on(Select.RecordCalls.retrieve.rets.collect())
    .aggregate(np.mean)
)

In [ ]:
from trulens_eval import TruCustomApp
tru_rag = TruCustomApp(rag,
    app_id = TestName,
    feedbacks = [f_groundedness, f_qa_relevance, f_context_relevance])

In [ ]:
#Avvio la registrazione dell'app utilizzando il dataset registrato
with tru_rag as recording:
    for i in range(DBEval.__len__()):
        if i == 0 & dataset_header:
            continue
        else:
            test = DBEval[i]
            ask = test[dataset_column_question]
            response = rag.query(ask, eval_row=i)

In [ ]:
tru.get_leaderboard(app_ids=[TestName])

In [ ]:
tru.run_dashboard()
#http://127.0.0.1:8501/